In [ ]:
# ipcluster start -n 4 --engines=MPIEngineSetLauncher

import ipyparallel as ipp
c = ipp.Client()
c.ids

In [ ]:
%%px
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

cartcomm_dims = int(np.sqrt(size))
cartcomm_dims = (cartcomm_dims, size // cartcomm_dims)

cartcomm = comm.Create_cart(cartcomm_dims, periods=(1, 1), reorder=False)
cartcomm_coords = cartcomm.coords
print('Rank {}/{}, Coords {}'.format(rank, size, cartcomm_coords))

In [ ]:
%%px
# Simulation settings (1.7; 300; 0.1)
w = 1.7 # Omega
domain_a = (300, 300) # Y,X domain_a

# Walls
wall = np.array([1, 1, 1, 1]) # Wall existence (E, N, W, S)
wall_speed = np.array([0, 0.1, 0, 0]) # Wall movement speed (E, N, W, S)

# Domain Decomposition
split = [domain_a[0] // cartcomm_dims[0], domain_a[1] // cartcomm_dims[1]]
if (cartcomm_coords[0] + 1) == cartcomm_dims[0]:
    split[0] = domain_a[0] - split[0] * cartcomm_coords[0]
if (cartcomm_coords[1] + 1) == cartcomm_dims[1]:
    split[1] = domain_a[1] - split[1] * cartcomm_coords[1]

# Domain Walls
if cartcomm_coords[0] != 0:
    wall[1] = 0
if (cartcomm_coords[0] + 1) != cartcomm_dims[0]:
    wall[3] = 0
if cartcomm_coords[1] != 0:
    wall[2] = 0
if (cartcomm_coords[1] + 1) != cartcomm_dims[1]:
    wall[0] = 0

print('Rank {}, Size {}, Wall ({}, {}, {}, {})'.format(rank, split, wall[0], wall[1], wall[2], wall[3]))

In [ ]:
%%px
# Constant arrays
w_c = np.array([4/9] + [1/9] * 4 + [1/36] * 4)
c_ac  = np.array([[0, 1, 0, -1, 0, 1, -1, -1, 1], [0, 0, 1, 0, -1, 1, 1, -1, -1]])
wall_c = np.array([[8, 1, 5], [5, 2, 6], [6, 3, 7], [7, 4, 8]]) # Walls with corresponding channels (E, N, W, S)
wall_f = np.array([-1/6, 0, +1/6]) # Wall speed factors
wall_p = np.array([[1, 2, 1, 0, 1, 2, 0, 0, 2], [1, 1, 2, 1, 0, 2, 2, 0, 0], [1, 0, 1, 2, 1, 0, 2, 2, 0], [1, 1, 0, 1, 2, 0, 0, 2, 2]]) # Wall Probability Weights (E, N, W, S)


# Communicate Data
def communicate(lattice_yxc):
    # Send right, receive left
    src, dst = cartcomm.Shift(1, 1)
    buffer = np.zeros(lattice_yxc[:, 0].shape)
    comm.Sendrecv(lattice_yxc[:, -2].copy(), dst, recvbuf=buffer, source=src)
    lattice_yxc[:, 0] = buffer

    # Send left, receive right
    src, dst = cartcomm.Shift(1, -1)
    comm.Sendrecv(lattice_yxc[:, 1].copy(), dst, recvbuf=buffer, source=src)
    lattice_yxc[:, -1] = buffer

    # Send up, receive down
    src, dst = cartcomm.Shift(0, -1)
    comm.Sendrecv(lattice_yxc[1, :], dst, recvbuf=lattice_yxc[-1, :], source=src)

    # Send down, receive up
    src, dst = cartcomm.Shift(0, 1)
    comm.Sendrecv(lattice_yxc[-2, :], dst, recvbuf=lattice_yxc[0, :], source=src)


# Stream the lattice channels, apply boundary conditions
def stream(lattice_yxc):
    # Calculate Wall Reflection
    if wall[0]: # East
        reflectE = lattice_yxc[ :,-2, wall_c[0]] - wall_speed[0] * np.einsum('yc, c, f -> yf', lattice_yxc[:, -2, :], wall_p[0], wall_f)
    if wall[1]: # North
        reflectN = lattice_yxc[ 1, :, wall_c[1]].T + wall_speed[1] * np.einsum('xc, c, f -> xf', lattice_yxc[1, :, :], wall_p[1], wall_f)
    if wall[2]: # West
        reflectW = lattice_yxc[ :, 1, wall_c[2]] + wall_speed[2] * np.einsum('yc, c, f -> yf', lattice_yxc[:, 1, :], wall_p[2], wall_f)
    if wall[3]: # South
        reflectS = lattice_yxc[-2, :, wall_c[3]].T - wall_speed[3] * np.einsum('xc, c, f -> xf', lattice_yxc[-2, :, :], wall_p[3], wall_f)

    # Stream Channels
    lattice_yxc[:, :, 1] = np.roll(lattice_yxc[:, :, 1], 1, axis=1) # Channel 1 [0, +1]
    lattice_yxc[:, :, 2] = np.roll(lattice_yxc[:, :, 2], -1, axis=0) # Channel 2 [-1, 0]
    lattice_yxc[:, :, 3] = np.roll(lattice_yxc[:, :, 3], -1, axis=1) # Channel 3 [0, -1]
    lattice_yxc[:, :, 4] = np.roll(lattice_yxc[:, :, 4], 1, axis=0) # Channel 4 [+1, 0]

    lattice_yxc[:, :, 5] = np.roll(lattice_yxc[:, :, 5], [-1, 1], axis=(0, 1)) # Channel 5 [-1, +1]
    lattice_yxc[:, :, 6] = np.roll(lattice_yxc[:, :, 6], [-1, -1], axis=(0, 1)) # Channel 6 [-1, -1]
    lattice_yxc[:, :, 7] = np.roll(lattice_yxc[:, :, 7], [1, -1], axis=(0, 1)) # Channel 7 [+1, -1]
    lattice_yxc[:, :, 8] = np.roll(lattice_yxc[:, :, 8], [1, 1], axis=(0, 1)) # Channel 8 [+1, +1]

    # Apply Wall Reflection
    if wall[0]: # East
        lattice_yxc[ :,-2, wall_c[2]] = reflectE
    if wall[2]: # West
        lattice_yxc[ :, 1, wall_c[0]] = reflectW
    if wall[3]: # South
        lattice_yxc[-2, :, wall_c[1]] = reflectS.T
    if wall[1]: # North
        lattice_yxc[ 1, :, wall_c[3]] = reflectN.T


# Calculate the physical properties (probability density values, velocitys) of the lattice
def physics(lattice_yxc):
    p_yx = np.einsum('yxc -> yx', lattice_yxc) # Calculate Probability Density
    v_yxa = np.einsum('yxc, ac, yx -> yxa', lattice_yxc, c_ac, 1/p_yx) # Calculate XY Speed

    return p_yx, v_yxa


# Calculate the equilibrium values of the lattice
def equilibrium(p_yx, v_yxa):
    # Splitted Equilibrium Formula
    feq_yxc = np.einsum('ac, yxa -> yxc', c_ac, v_yxa)
    feq_yxc = 1 + 3 * feq_yxc + 9/2 * np.square(feq_yxc) - 3/2 * np.sum(np.square(v_yxa), axis=2)[:, :, np.newaxis]
    feq_yxc = np.einsum('c, yx, yxc -> yxc', w_c, p_yx, feq_yxc)

    return feq_yxc


# Update lattice according to the choosen viscosity and the calculated equilibrium values
def collision(lattice_yxc):
    p_yx, v_yxa = physics(lattice_yxc)
    feq_yxc = equilibrium(p_yx, v_yxa)
    
    # Update Step
    lattice_yxc += w * (feq_yxc - lattice_yxc)

    return p_yx, v_yxa

In [ ]:
%%px
# Execute Simulation
steps = (1000001, 10000)
def run(lattice_yxc):
    x_i, y_i = np.linspace(0, domain_a[1], domain_a[1]), np.linspace(0, domain_a[0], domain_a[0]) # Visualization Coordinates

    for i in range(steps[0]):
        communicate(lattice_yxc)
        stream(lattice_yxc)
        p_yx, v_yxa = collision(lattice_yxc)

        # Visualize
        if True and i % steps[1] == 0:
            if rank == 0:
                lattice_full = np.zeros((domain_a[0] * cartcomm_dims[1], split[0], 9))
                comm.Gather(lattice_yxc[1:-1, 1:-1].copy(), lattice_full, root=0)

                # Rearange Gathered Data
                lattice_full = np.split(lattice_full, lattice_full.shape[0]/split[0], axis=0)
                lattice_full = np.concatenate(lattice_full, axis=1)
                lattice_full = np.split(lattice_full, lattice_full.shape[1]/domain_a[1], axis=1)
                lattice_full = np.concatenate(lattice_full, axis=0)

                if False: # Show Result
                    if i == 0:
                        import matplotlib.pyplot as plt
                        %matplotlib inline

                    p_full, v_full = physics(lattice_full)

                    print(np.sum(p_full))
                    plt.figure(figsize=(10, int((domain_a[1] / domain_a[0]) * 10)))
                    canvas = plt.subplot()

                    canvas.imshow(p_full, vmin=0, vmax=2)
                    canvas.streamplot(x_i, y_i, v_full[:,:,0], -v_full[:,:,1], color='white')

                    canvas.get_xaxis().set_visible(False)
                    canvas.get_yaxis().set_visible(False)

                    plt.show()
                else: # Save Data
                    np.save('lattice_{}'.format(i), lattice_full)
            else:
                comm.Gather(lattice_yxc[1:-1, 1:-1].copy(), None, root=0)

# Local Lattice
p_yx = np.ones((split[0] + 2, split[1] + 2))
v_yxa = np.zeros((split[0] + 2, split[1] + 2, 2))

lattice_yxc = equilibrium(p_yx, v_yxa)

run(lattice_yxc)